In [0]:
from datetime import date, timedelta

# Remove this before running Data Factory Pipeline
start_date = date.today() - timedelta(1)

bronze_adls = "abfss://bronze@eqstoreacc.dfs.core.windows.net/"
silver_adls = "abfss://silver@eqstoreacc.dfs.core.windows.net/"

In [0]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

In [0]:
df.head()

Row(geometry=Row(coordinates=[-122.810836791992, 38.8411674499512, 1.96000003814697], type='Point'), id='nc75219247', properties=Row(alert=None, cdi=None, code='75219247', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75219247&format=geojson', dmin=0.01624, felt=None, gap=52.0, ids=',nc75219247,', mag=1.36, magType='md', mmi=None, net='nc', nst=24, place='8 km WNW of Cobb, CA', rms=0.02, sig=28, sources=',nc,', status='automatic', time=1754351979090, title='M 1.4 - 8 km WNW of Cobb, CA', tsunami=0, type='earthquake', types=',nearby-cities,origin,phase-data,', tz=None, updated=1754352076665, url='https://earthquake.usgs.gov/earthquakes/eventpage/nc75219247'), type='Feature')

In [0]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [0]:
df.head()

Row(id='nc75219247', longitude=-122.810836791992, latitude=38.8411674499512, elevation=1.96000003814697, title='M 1.4 - 8 km WNW of Cobb, CA', place_description='8 km WNW of Cobb, CA', sig=28, mag=1.36, magType='md', time=1754351979090, updated=1754352076665)

In [0]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [0]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [0]:
df.head()

Row(id='nc75219247', longitude=-122.810836791992, latitude=38.8411674499512, elevation=1.96000003814697, title='M 1.4 - 8 km WNW of Cobb, CA', place_description='8 km WNW of Cobb, CA', sig=28, mag=1.36, magType='md', time=datetime.datetime(2025, 8, 4, 23, 59, 39, 90000), updated=datetime.datetime(2025, 8, 5, 0, 1, 16, 665000))

In [0]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"
     


In [0]:

# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)